Возьмите англо-русскую пару фраз (https://www.manythings.org/anki/)
Обучите на них seq2seq по аналогии с занятием. Оцените полученное качество
Попробуйте добавить +1 рекуррентный слой в encoder и decoder
Попробуйте заменить GRU ячейки на lstm-ячейки
Оцените качество во всех случаях
*5. Добавит в лекционный ноутбук, в функцию train, обучение батчами.

In [ ]:
%matplotlib inline

In [ ]:
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!wget https://www.manythings.org/anki/rus-eng.zip
!unzip rus-eng.zip

--2024-12-23 11:13:40--  https://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16305013 (16M) [application/zip]
Saving to: ‘rus-eng.zip’

rus-eng.zip         100%[===================>]  15.55M  46.0MB/s    in 0.3s    

2024-12-23 11:13:41 (46.0 MB/s) - ‘rus-eng.zip’ saved [16305013/16305013]

Archive:  rus-eng.zip
  inflating: rus.txt                 
  inflating: _about.txt              


In [ ]:
!tail rus.txt

We need to uphold laws against discrimination — in hiring, and in housing, and in education, and in the criminal justice system. That is what our Constitution and our highest ideals require.	Нам нужно отстаивать законы против дискриминации при найме на работу, в жилищной сфере, в сфере образования и правоохранительной системе. Этого требуют наша Конституция и высшие идеалы.	CC-BY 2.0 (France) Attribution: tatoeba.org #5762728 (BHO) & #6390439 (odexed)
I've heard that you should never date anyone who is less than half your age plus seven. Tom is now 30 years old and Mary is 17. How many years will Tom need to wait until he can start dating Mary?	Я слышал, что никогда не следует встречаться с кем-то вдвое младше вас плюс семь лет. Тому 30 лет, a Мэри 17. Сколько лет Тому нужно ждать до тех пор, пока он сможет начать встречаться с Мэри?	CC-BY 2.0 (France) Attribution: tatoeba.org #10068197 (CK) & #10644473 (notenoughsun)
I do have one final ask of you as your president, the same thing I a

In [ ]:
!head rus.txt

Go.	Марш!	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1159202 (shanghainese)
Go.	Иди.	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #5898247 (marafon)
Go.	Идите.	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #5898250 (marafon)
Hi.	Здравствуйте.	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #402127 (odexed)
Hi.	Привет!	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #466968 (katjka)
Hi.	Хай.	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #467233 (timsa)
Hi.	Здрасте.	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #3803577 (marafon)
Hi.	Здоро́во!	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #3854188 (marafon)
Hi.	Приветик!	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #7234283 (marafon)
Run!	Беги!	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #1569978 (Biga)


Чтобы сделать то что было на олекции, сначалал нжно подготовить данные: удалить лишние столбцы.

In [ ]:
import pandas as pd
df = pd.read_csv('rus.txt', sep ='\t', header=None)
df = df.iloc[:, :2]
df = df.iloc[::10] # берем каждое 10 значение.
df = df.reset_index(drop=True)
df.head()
df.to_csv('rus.txt', index=False, header=False, sep='\t') # запись  в то же файл

In [ ]:
df.head(3)

,0,1
0,Go.,Марш!
1,Run!,Бегите!
2,Duck!,Пригнись!


In [ ]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?А-Яа-я]+", r" ", s)
    return s

In [ ]:
lines = open('rus.txt', encoding='utf-8').read().split('\n')
lines[0]

'Go.\tМарш!'

In [ ]:
lines[len(lines)-1]

''

In [ ]:
lines[len(lines)-2]

"I've heard that you should never date anyone who is less than half your age plus seven. Tom is now 30 years old and Mary is 17. How many years will Tom need to wait until he can start dating Mary?\tЯ слышал, что никогда не следует встречаться с кем-то вдвое младше вас плюс семь лет. Тому 30 лет, a Мэри 17. Сколько лет Тому нужно ждать до тех пор, пока он сможет начать встречаться с Мэри?"

In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('rus.txt', encoding='utf-8').read().split('\n')
    lines.pop() # удаление последнего индекса =''
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

[['j ai ans .', 'i m .'],
 ['je vais bien .', 'i m ok .'],
 ['ca va .', 'i m ok .'], что на выходе должно получиться.

In [ ]:
 input_lang, output_lang, pairs = readLangs('rus','eng', True)
pairs[:5]

Reading lines...


[['марш !', 'go .'],
 ['бегите !', 'run !'],
 ['пригнись !', 'duck !'],
 ['спрячьтесь .', 'hide .'],
 ['остановитесь !', 'stop !']]

In [ ]:
MAX_LENGTH = 10 # вводим ограничение на длину предложения не больше 10

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
len(pairs)

49606

In [ ]:
pairs[len(pairs)-1]

['я слышал что никогда не следует встречаться с кем то вдвое младше вас плюс семь лет . тому лет a мэри . сколько лет тому нужно ждать до тех пор пока он сможет начать встречаться с мэри ?',
 'i ve heard that you should never date anyone who is less than half your age plus seven . tom is now years old and mary is . how many years will tom need to wait until he can start dating mary ?']

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'ru', True)
print(random.choice(pairs))

Reading lines...
Read 49606 sentence pairs
Trimmed to 40903 sentence pairs
Counting words...
Counted words:
ru 18191
eng 7322
['я сделал это когда был пьян .', 'i did that when i was drunk .']


The Encoder
-----------





In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder
-----------




In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

1m 22s (- 19m 20s) (5000 6%) 4.3197
2m 36s (- 16m 55s) (10000 13%) 3.8284
3m 50s (- 15m 20s) (15000 20%) 3.5465
5m 8s (- 14m 9s) (20000 26%) 3.3271
6m 22s (- 12m 45s) (25000 33%) 3.1664
7m 42s (- 11m 33s) (30000 40%) 3.0510
8m 55s (- 10m 11s) (35000 46%) 2.9064
10m 8s (- 8m 52s) (40000 53%) 2.7920
11m 22s (- 7m 35s) (45000 60%) 2.7811
12m 36s (- 6m 18s) (50000 66%) 2.6911
13m 54s (- 5m 3s) (55000 73%) 2.6321
15m 7s (- 3m 46s) (60000 80%) 2.5842
16m 21s (- 2m 31s) (65000 86%) 2.5017
17m 35s (- 1m 15s) (70000 93%) 2.4412
18m 49s (- 0m 0s) (75000 100%) 2.4223


In [ ]:
evaluateRandomly(encoder1, decoder1) # предсказывает отдельные слова, но не всегда хорошо.

> я люблю их обоих .
= i love both of them .
< i love them them . <EOS>

> несчастныи случаи произошел на этом перекрестке .
= the accident happened at that intersection .
< the the is the to . . <EOS>

> том не может произнести фамилию мэри .
= tom can t pronounce mary s last name .
< tom didn t want mary to mary . . <EOS>

> мне жаль что я не могу тебе помочь .
= i regret that i can t help you .
< i wish i can t trust you you . .

> мы даже не знаем где мы .
= we don t even know where we are .
< we even know where we are . . <EOS>

> мои сын всего лишь подросток .
= my son is only a teenager .
< my son is a son . <EOS>

> мне сегодня немного грустно .
= i m feeling a little sad today .
< i have a little bit today . <EOS>

> я вас за это не прощу .
= i won t forgive you for this .
< i m sorry i won t do it . <EOS>

> хорошо что вы не поехали .
= it was good that you didn t go .
< i hope you re t good good well . <EOS>

> расскажи мне немного о себе .
= tell me a bit about yourself .
<

Попробуйте добавить +1 рекуррентный слой в encoder и decoder

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru1 = nn.GRU(hidden_size, hidden_size)
        self.gru2 = nn.GRU(hidden_size, hidden_size)# добавим еще слой

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded


        output, hidden = self.gru1(output, hidden) #1 проход
        output, hidden = self.gru2(output, hidden) # 2 проход
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru1 = nn.GRU(hidden_size, hidden_size)
        self.gru2 = nn.GRU(hidden_size, hidden_size) # добавим еще слой
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        # Прямой проход через первый GRU
        output, hidden = self.gru1(output, hidden)

        # Прямой проход через второй GRU
        output, hidden = self.gru2(output, hidden)

        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

2m 4s (- 29m 6s) (5000 6%) 4.4289
4m 4s (- 26m 30s) (10000 13%) 4.1321
6m 1s (- 24m 6s) (15000 20%) 3.9614
7m 57s (- 21m 52s) (20000 26%) 3.8162
9m 54s (- 19m 49s) (25000 33%) 3.6221
11m 51s (- 17m 46s) (30000 40%) 3.4856
13m 48s (- 15m 47s) (35000 46%) 3.3280
15m 45s (- 13m 47s) (40000 53%) 3.2283
17m 41s (- 11m 47s) (45000 60%) 3.1368
19m 38s (- 9m 49s) (50000 66%) 3.0439
21m 34s (- 7m 50s) (55000 73%) 2.9489
23m 31s (- 5m 52s) (60000 80%) 2.8689
25m 27s (- 3m 55s) (65000 86%) 2.8408
27m 24s (- 1m 57s) (70000 93%) 2.7878
29m 20s (- 0m 0s) (75000 100%) 2.7025


 Попробуйте заменить GRU ячейки на lstm-ячейки

In [ ]:
evaluateRandomly(encoder1, decoder1) # потери больше чем в пред. модели. По виду модели видно, что качество ниже.

> я готова .
= i am ready .
< i m a . <EOS>

> я был очень очень счастлив .
= i was very very happy .
< i was very very . . . <EOS>

> я устал и мне было холодно .
= i was tired and cold .
< i was t to to me to . . <EOS>

> у тебя есть во что переодеться ?
= do you have anything to change into ?
< do you have anything you have ? ? ? <EOS>

> когда то здесь был большои парк .
= there used to be a large park here .
< it was be here here . here . <EOS>

> том смотрит хоккеиныи матч .
= tom is watching a hockey game .
< tom is going to be a . . <EOS>

> у вас есть с собои авторучка ?
= do you have a fountain pen with you ?
< do you have a bike ? <EOS>

> вам нужно перезагрузить компьютер .
= you need to restart your computer .
< you need to be a . . . <EOS>

> я тебе надоел всем этим ?
= am i boring you with all this ?
< did i tell you that for this ? <EOS>

> я знаю обоих братьев тома .
= i know both of tom s brothers .
< i know tom s tom . tom . <EOS>



In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, (hidden, cell) = self.lstm(embedded, hidden)  # Передаем кортеж состояний
        return output, (hidden, cell)

    def initHidden(self):
        hidden = torch.zeros(1, 1, self.hidden_size, device=device)
        cell = torch.zeros(1, 1, self.hidden_size, device=device)
        return (hidden, cell)

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)  # Заменили GRU на LSTM
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)

        # Передаем скрытое состояние и состояние ячейки
        output, (hidden, cell) = self.lstm(output, hidden)  # Передаем кортеж состояний
        output = self.softmax(self.out(output[0]))
        return output, (hidden, cell)  # Возвращаем оба состояния

    def initHidden(self):
        # Инициализация скрытого состояния и состояния ячейки
        hidden = torch.zeros(1, 1, self.hidden_size, device=device)
        cell = torch.zeros(1, 1, self.hidden_size, device=device)
        return (hidden, cell)  # Возвращаем кортеж из скрытого состояния и состояния ячейки

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

1m 24s (- 19m 48s) (5000 6%) 4.4241
2m 45s (- 17m 57s) (10000 13%) 3.9962
4m 8s (- 16m 32s) (15000 20%) 3.7641
5m 30s (- 15m 9s) (20000 26%) 3.5611
6m 52s (- 13m 44s) (25000 33%) 3.4108
8m 14s (- 12m 22s) (30000 40%) 3.2771
9m 37s (- 11m 0s) (35000 46%) 3.1652
10m 59s (- 9m 37s) (40000 53%) 3.0741
12m 22s (- 8m 15s) (45000 60%) 2.9747
13m 45s (- 6m 52s) (50000 66%) 2.9009
15m 8s (- 5m 30s) (55000 73%) 2.8307
16m 31s (- 4m 7s) (60000 80%) 2.7768
17m 54s (- 2m 45s) (65000 86%) 2.7299
19m 17s (- 1m 22s) (70000 93%) 2.6752
20m 40s (- 0m 0s) (75000 100%) 2.6233


In [ ]:
evaluateRandomly(encoder1, decoder1) # потери здесь немного меньше чем  с 2 скрытыми слоями. Отдельные слова переводи.

> это несложно !
= it s not hard .
< it s a . <EOS>

> я не дам вам денег .
= i won t give you money .
< i don t need you . <EOS>

> я забыл оставить официанту чаевые .
= i forgot to tip the waiter .
< i ve decided to leave the . <EOS>

> думаешь том может это наити ?
= do you think tom can find it ?
< do you think tom will help ? <EOS>

> не уходите пока домои .
= don t go home yet .
< don t leave us . <EOS>

> ты не похож на наемного убиицу .
= you don t look like a killer .
< you re not like your a . . <EOS>

> вы солдаты ?
= are you soldiers ?
< are you going to be ? <EOS>

> я был глупым .
= i was stupid .
< i was was . <EOS>

> человек чистит зубы .
= the man brushes his teeth .
< the is is the . <EOS>

> твои дядя сделал эти фотографии не так ли ?
= your uncle took these pictures didn t he ?
< these these these these these these this isn t ?

